In [1]:
import pandas as pd
import numpy as np

# 1035, 258
train = pd.read_csv('/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/d-optimal/spmm-latency-traintest/train-test-csv/nonsquare-train-1035-from-spmm-contain-todense-over-3s-1293.csv')
test = pd.read_csv('/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/d-optimal/spmm-latency-traintest/train-test-csv/nonsquare-test-258-from-spmm-contain-todense-over-3s-1293.csv')

In [2]:
# Train + Valid
X_train = train[['lr','lc','rc','ld','rd','lnnz','rnnz','lr*lc','lc*rc','lr*rc','lr*lc*rc','ld*rd','lr*rc*ld*rd','lr*lc*rc*ld*rd','lnnz*rnnz']] 
y_train = train['sp_smdm']

# Test
X_test = test[['lr','lc','rc','ld','rd','lnnz','rnnz','lr*lc','lc*rc','lr*rc','lr*lc*rc','ld*rd','lr*rc*ld*rd','lr*lc*rc*ld*rd','lnnz*rnnz']] 
y_test = test['sp_smdm']

In [3]:
from sklearn.metrics import r2_score, mean_squared_error

# Metric
def mean_absolute_percentage_error(y_test, y_pred):
    y_test, y_pred = np.array(y_test), np.array(y_pred)
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

def custom_scoring(real, pred):
    rmse = np.sqrt(mean_squared_error(real, pred))
    r2 = r2_score(real, pred)
    mape = mean_absolute_percentage_error(real, pred)
    return mape

In [4]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer

# 탐색 대상 함수 (XGBRegressor)
def XGB(
max_depth,
learning_rate, 
n_estimators, 
min_child_weight, 
subsample,
_lambda,
gamma ,
colsample_bytree, 
_alpha,
silent=True, 
n_jobs=-1):
    
    # 모델 정의
    model = xgb.XGBRegressor( 
objective = 'reg:squarederror',
max_depth=int(max_depth),
learning_rate=learning_rate,
n_estimators=int(n_estimators),
min_child_weight=min_child_weight,
subsample=subsample,
reg_lambda=_lambda,        
gamma=gamma,
colsample_bytree=colsample_bytree, 
reg_alpha=_alpha,
n_jobs=n_jobs        
                              )
    
    # bayesian optimization을 통해 파라미터를 받아
    # Train을 Train + Validation으로 나눠 cross-validation 성능 확인
    kfold = KFold(n_splits=12, shuffle = True, random_state=0)

    # cross-validation 평균 성능 성능 확인
    score = cross_val_score(model,
                            X_train, 
                            y_train, 
                            cv=kfold,
                            scoring=make_scorer(custom_scoring,greater_is_better=False),
                            n_jobs=-1
                           ).mean()
    
    return score

In [5]:
from bayes_opt import BayesianOptimization

# # 하이퍼파라미터 정의
# pbounds = {
# 'max_depth': (12, 16), 
# 'learning_rate': (0.01, 0.3),
# 'n_estimators': (50, 150),
# 'min_child_weight': (0.1, 1),
# 'subsample': (0.5, 1), 
# '_lambda' : (0.1,1),    
# #'gamma': (0, 0.3),     
# #'colsample_bytree' :(0.5, 1)
# #'_alpha' : (0,1)           
#                       }

# 하이퍼파라미터 정의
pbounds = {
'max_depth': (5, 10), 
'learning_rate': (0.01, 0.3),
'n_estimators': (6000, 7000),
'min_child_weight': (0.1, 1),
'subsample': (0.5, 1), 
'_lambda' : (0.1,1),    
'gamma': (0, 0.6),     
'colsample_bytree' :(0.5, 1),
'_alpha' : (0,0.1)
                      }


# Bayesian optimization 객체 생성
bo=BayesianOptimization(f=XGB, pbounds=pbounds, verbose=2, random_state=1 )    

# 메소드를 이용해 최대화 과정 수행 (파라미터 넣고 목적함수 값 출력하고)
bo.maximize(init_points=2, n_iter=50, acq='ei', xi=0.01)

# 뽑힌 최적의 하이퍼파라미터 값 확인
print("{}\n".format(bo.max))

|   iter    |  target   |  _alpha   |  _lambda  | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------
|  1        | -24.14    |  0.0417   |  0.7483   |  0.5001   |  0.1814   |  0.05256  |  5.462    |  0.2676   |  6.346e+0 |  0.6984   |
|  2        | -24.54    |  0.05388  |  0.4773   |  0.8426   |  0.1227   |  0.2647   |  5.137    |  0.7034   |  6.417e+0 |  0.7793   |
|  3        | -20.36    |  0.01747  |  1.0      |  0.5      |  0.358    |  0.01     |  6.257    |  0.1      |  6.132e+0 |  0.533    |
|  4        | -22.44    |  0.08453  |  1.0      |  0.9006   |  0.5573   |  0.2415   |  6.688    |  0.8169   |  6.1e+03  |  0.9069   |
|  5        | -24.79    |  0.01514  |  0.9338   |  0.5273   |  0.4091   |  0.0911   |  6.621    |  0.2303   |  6.132e+0 |  0.5929   |
|  6        | -19.58    |  0.05026  |  0.7708   |  0.6702   | 

/opt/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


|  8        | -20.26    |  0.09268  |  0.9652   |  0.6934   |  0.4552   |  0.1024   |  9.089    |  0.7103   |  6.313e+0 |  0.5961   |
|  9        | -23.27    |  0.09514  |  0.6014   |  0.9215   |  0.1036   |  0.2447   |  6.712    |  0.7741   |  6.835e+0 |  0.6578   |
|  10       | -19.25    |  0.06104  |  0.1406   |  0.7181   |  0.2825   |  0.04649  |  6.577    |  0.3175   |  6.84e+03 |  0.9259   |
|  11       | -22.11    |  0.00288  |  0.2896   |  0.8429   |  0.391    |  0.2904   |  8.684    |  0.1916   |  6.932e+0 |  0.7577   |
|  12       | -20.29    |  0.01901  |  0.4127   |  0.9913   |  0.1735   |  0.139    |  7.139    |  0.9187   |  6.829e+0 |  0.9934   |
|  13       | -24.06    |  0.04124  |  0.1578   |  0.5111   |  0.5357   |  0.05952  |  5.338    |  0.1336   |  6.958e+0 |  0.6605   |
|  14       | -19.44    |  0.0791   |  0.4886   |  0.6954   |  0.3489   |  0.09063  |  8.318    |  0.7224   |  6.132e+0 |  0.7147   |
|  15       | -24.12    |  0.03705  |  0.3673   |  1.0      | 

|  45       | -21.49    |  0.02863  |  0.2351   |  0.8934   |  0.04734  |  0.1661   |  6.295    |  0.9108   |  6.077e+0 |  0.9761   |
|  46       | -21.66    |  0.0185   |  0.4518   |  0.8934   |  0.3766   |  0.1525   |  7.865    |  0.2252   |  6.133e+0 |  0.522    |
|  47       | -20.5     |  0.02351  |  0.241    |  0.6965   |  0.2244   |  0.07838  |  9.64     |  0.6006   |  6.201e+0 |  0.9675   |
|  48       | -18.0     |  0.02913  |  0.5073   |  0.9795   |  0.5367   |  0.05994  |  7.494    |  0.5586   |  6.655e+0 |  0.7248   |
|  49       | -19.26    |  0.03599  |  0.5022   |  0.7268   |  0.3351   |  0.07388  |  8.687    |  0.3116   |  6.887e+0 |  0.9469   |
|  50       | -17.89    |  0.09904  |  0.9815   |  0.8334   |  0.289    |  0.03444  |  9.349    |  0.6874   |  6.108e+0 |  0.579    |
|  51       | -19.38    |  0.07703  |  0.7895   |  0.9144   |  0.07431  |  0.03552  |  5.076    |  0.973    |  6.317e+0 |  0.967    |


KeyboardInterrupt: 

In [ ]:
import xgboost as xgb

# Train + Valid cross-validation을 거친, 최적의 하이퍼파라미터를 사용
best_model = xgb.XGBRegressor(
objective = 'reg:squarederror',
max_depth=int(bo.max['params']['max_depth']),
learning_rate=bo.max['params']['learning_rate'],
n_estimators=int(bo.max['params']['n_estimators']),
min_child_weight=bo.max['params']['min_child_weight'],
subsample=bo.max['params']['subsample'],
reg_lambda = bo.max['params']['_lambda'],    
gamma=bo.max['params']['gamma'],
colsample_bytree=bo.max['params']['colsample_bytree'],
reg_alpha = bo.max['params']['_alpha'],
n_jobs=-1
                             )
# 모델 훈련
best_model.fit(X_train, y_train)

In [ ]:
# 최적의 하이퍼파라미터 값 확인
print("{}\n".format(bo.max))

# 훈련데이터 예측
y_train_pred = best_model.predict(X_train)
print("-------- 훈련데이터 예측 --------------------------")
print("rmse : {}".format(np.sqrt(mean_squared_error(y_train, y_train_pred))))
print("mape : {}%".format(mean_absolute_percentage_error(y_train, y_train_pred)))
print("\n")

# 검증데이터 예측
print("-------- 검증데이터 예측 --------------------------")
print("mape : {}%".format(-bo.max['target']))
print("\n")

# 테스트데이터 예측
y_pred = best_model.predict(X_test)
print("-------- 테스트데이터 예측 -------------------------")
print("rmse : {}".format(np.sqrt(mean_squared_error(y_test, y_pred))))
print("mape : {}%".format(mean_absolute_percentage_error(y_test, y_pred)))
print("\n")

In [ ]:
# mape_list = {}
# # 예측값, 실제값을 확인하며 mape 계산 후 mape_list에 삽입 
# for idx,value in enumerate(y_test):
#     mape_temp = {}
#     predicate = int(y_pred[idx])
#     mape = abs((value - predicate) / value) * 100
#     mape_temp['pred'] = predicate
#     mape_temp['real'] = value
#     mape_temp['mape'] = mape
#     mape_list[idx] = mape_temp
# mape_list_sort = sorted(mape_list.values(), key=lambda x:(x['mape']), reverse=True)
# mape_list_sort  

In [ ]:
xgb.plot_importance(best_model)